In [ ]:
from types import SimpleNamespace
import numpy as np
import deepdish as dd
import os
import holoviews as hv
import funcs
import pandas as pd
hv.notebook_extension()
%opts Scatter (color='b') 
d = SimpleNamespace(phi=hv.Dimension(r'$\phi$', unit='rad'),
                    phi_c=hv.Dimension(r'$\phi_c$', unit='rad'),
                    I=hv.Dimension('$I$', unit=r'meV / $\Phi_0$'),
                    I_c=hv.Dimension('$I_c$', unit=r'meV / $\Phi_0$'),
                    B=hv.Dimension('$B$', unit='T'),
                    T=hv.Dimension('$T$', unit='K'), 
                    L=hv.Dimension('$L$', unit='nm'), 
                    g=hv.Dimension('$g$-factor'), 
                    alpha=hv.Dimension(r'$\alpha$', unit='mev nm'), 
                    mu=hv.Dimension(r'$\mu$', unit='meV'), 
                    a=hv.Dimension('$a$', unit='nm'),
                    V=hv.Dimension('$V$', unit='meV'))

pd.set_option('display.max_columns', None)

# Disorder

In [ ]:
path = './'
files = [f for f in os.listdir(path) if f.startswith('disorder_B_vs_I_c')]

fs = {f: dd.io.load(path + f) for f in files}

hm = {}

    
for fname, data in fs.items():
    disorders = data['disorders']
    I_cs = data['I_cs']
    Bs = data['B_xs']
    wire_params = data['wire_params']
    p = data['p']
    dims = dict(kdims=[hv.Dimension('$B$', unit='$T$')], vdims=['$I_c$'])

    for disorder, I_cs, std in zip(disorders, I_cs.mean(axis=0), I_cs.std(axis=0)):
        key = (wire_params['a'], wire_params['L'], data['T'], 
               p.mu, disorder, wire_params['with_leads'], p.orbital, p.g)
        val = hv.Spread((Bs, I_cs, std), kdims=[hv.Dimension('$B$', unit='$T$')], vdims=[hv.Dimension('$I_c$'), 'err'])
        val *= hv.Path((Bs, I_cs), kdims=[hv.Dimension('$B$', unit='$T$'), '$I_c$'])
        hm[key] = val[:, 0:100]


hv.HoloMap(hm, kdims=[hv.Dimension('$a$', unit='nm'),
                      hv.Dimension('$L$', unit='nm'),
                      hv.Dimension('$T$', unit='K'),
                      hv.Dimension('$\mu$', unit='meV'),
                      hv.Dimension('disorder', unit='meV'),
                      'infinite leads', 'orbital', 'zeeman']).select(T=1)

In [ ]:
len(files)

# Mean free path $g \sim N_{ch} / (1 + L / \lambda_{MFP})$
Check with Carlo's review paper on random matrix theory

In [ ]:
df = pd.read_hdf('data/mean_free_path.hdf')
gb = df.groupby(['disorder', 'L', 'mu', 'num_propagating'])['transmission']
conduction = gb.agg({'means' : np.mean, 'vars' : np.var}).reset_index()

d = {}
for key, gr in conduction.groupby(['L', 'mu']):
    d[key] = (hv.Curve((gr.disorder, gr.means), kdims=['disorder'], vdims=['resistance']) *
              hv.Spread((gr.disorder, gr.means, gr.vars)) *
              hv.Scatter((gr.disorder, gr.means)))[:, 0:25]
hv.HoloMap(d, kdims=['L', 'mu'])

In [ ]:
for (mu, disorder), gr in conduction.groupby(['mu', 'disorder']):
    modes = gr.num_propagating.values[0]
    fit = np.polyfit(gr.L, 1/gr.means, deg=1)
    mfp = 1 / (modes * fit[0])
    print("mu: {} meV, disorder: {} meV, mfp: {} nm".format(mu, disorder, mfp))